**ENSEMBLE MODEL: STACKING**

**DATA LOADING AND PREPARATION**

In [1]:
import pandas as pd

In [2]:
# Read the CSV
tfidf_df = pd.read_csv("tfidf_sncb.csv", sep='\,', engine='python')

tfidf_df['incident_type'] = tfidf_df['incident_type'].astype('string') 

tfidf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011 entries, 0 to 1010
Columns: 828 entries, incident_id to 998
dtypes: float64(826), int64(1), string(1)
memory usage: 6.4 MB


In [3]:
from sklearn.model_selection import train_test_split

# Filter in the Features (the values acquired from the events sequence after TF-IDF)
X = tfidf_df.drop(['incident_type', 'incident_id'], axis=1) 

# Filter in the Target variable (labels / incident types)
y = tfidf_df['incident_type']  

# setting random_state constant to be used in the whole pipeline and guarantee reproducibility
r_state = 123

# Split data into training+validation and testing sets
train_val_X, test_X, train_val_y, test_y = train_test_split(X, 
                                                            y, 
                                                            train_size = 0.8, 
                                                            random_state = r_state, # setting random_state for reproducibility
                                                            stratify = y) # to respect class imbalance in the label column

print(f"The train_val_X pandas df has {len(train_val_X)} rows and {len(train_val_X.columns)} columns.")
print(f"The test_y pandas series has {len(test_y)} rows and 1 column.")

The train_val_X pandas df has 808 rows and 826 columns.
The test_y pandas series has 203 rows and 1 column.


In [4]:
from collections import Counter

# get the size of the smallest incident type class
value_counts = Counter(train_val_y)
min_class_setsize = min(value_counts.values())

print(f"In RepeatedStratifiedKFold() function, the parameter n_splits has to be set atmost to {min_class_setsize}, due to class imbalance in the label column.")

In RepeatedStratifiedKFold() function, the parameter n_splits has to be set atmost to 3, due to class imbalance in the label column.


**MODEL TRAINING AND VALIDATION**

**SETUP**

In [5]:
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from datetime import datetime

# Define array of base models
base_models = [
    ('Random Forest', RandomForestClassifier(random_state = r_state, n_jobs = -1,
                                             criterion = "gini",
                                             max_features = 0.12, 
                                             class_weight = None)),
    ('Neural Networks', MLPClassifier(random_state = r_state,
                                      solver = "lbfgs",
                                      hidden_layer_sizes=(100,))),
    ('Gradient Boosting', GradientBoostingClassifier(random_state = r_state,
                                                     loss = "log_loss",
                                                     criterion = "friedman_mse",
                                                     max_features = 0.12))
]

# Set up inner cross-validation of model_stacking
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=r_state)

# Define Stacking
model_stacking = StackingClassifier(estimators = base_models, 
                                    final_estimator = LogisticRegression(),
                                    cv = skf,
                                    n_jobs = -1,
                                    verbose = 1
                                   )

# Set up cross-validation
rskf = RepeatedStratifiedKFold(n_splits = min_class_setsize, 
                               n_repeats = 34, 
                               random_state = r_state)

**TRAINING AND VALIDATION**

In [6]:
# Start timing
start_time = datetime.now()

# Initialize lists to store metrics
accuracy_scores = []
weighted_f1_scores = []
micro_f1_scores = []
macro_f1_scores = []
fold = 1

# Cross-validation loop
for train_idx, val_idx in rskf.split(train_val_X, train_val_y):

    # Uncomment the print statement for debugging only
    print(f"Starting model training and validation on fold {fold}")
    
    # Split data
    fold_train_X, fold_val_X = train_val_X.iloc[train_idx], train_val_X.iloc[val_idx]
    fold_train_y, fold_val_y = train_val_y.iloc[train_idx], train_val_y.iloc[val_idx]

    # Train the ensemble model
    model_stacking.fit(fold_train_X, fold_train_y)  # Train the ensemble model
    
    # Predict on the test set
    fold_pred_y = model_stacking.predict(fold_val_X)
    
    # Compute metrics
    accuracy_scores.append(accuracy_score(fold_val_y, fold_pred_y))
    weighted_f1_scores.append(f1_score(fold_val_y, fold_pred_y, average='weighted'))
    micro_f1_scores.append(f1_score(fold_val_y, fold_pred_y, average='micro'))
    macro_f1_scores.append(f1_score(fold_val_y, fold_pred_y, average='macro'))
    
    fold += 1

# End timing
end_time = datetime.now()
elapsed_time = end_time - start_time

# Aggregate results
print(f"The training and validation of model_stacking took {elapsed_time.total_seconds():.2f} seconds across {rskf.get_n_splits()} iterations.\n")

print(f"Mean Accuracy:          {np.mean(accuracy_scores):.8f} ± {np.std(accuracy_scores):.8f}")
print(f"Mean Weighted F1-Score: {np.mean(weighted_f1_scores):.8f} ± {np.std(weighted_f1_scores):.8f}")
print(f"Mean Micro F1-Score:    {np.mean(micro_f1_scores):.8f} ± {np.std(micro_f1_scores):.8f}")
print(f"Mean Macro F1-Score:    {np.mean(macro_f1_scores):.8f} ± {np.std(macro_f1_scores):.8f}\n")

print(f"Each fold had {len(fold_train_X)} entries for training and {len(fold_val_X)} for validation.")

Starting model training and validation on fold 1


/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
/usr/local/Caskroom/min

Starting model training and validation on fold 2


/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
[Parallel(n_jobs=-1)]: 

Starting model training and validation on fold 3


...

Starting model training and validation on fold 101


/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: 

Starting model training and validation on fold 102


/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: 

The training and validation of model_stacking took 1064.65 seconds across 102 iterations.

Mean Accuracy:          0.68550456 ± 0.02330337
Mean Weighted F1-Score: 0.66898962 ± 0.02293799
Mean Micro F1-Score:    0.68550456 ± 0.02330337
Mean Macro F1-Score:    0.35134204 ± 0.01969395

Each fold had 539 entries for training and 269 for validation.


**WITH BASE MODELS OPTIMIZATION AND CV = RSK(3, SHUFFLE=TRUE)**

**The training and validation of model_stacking took 995.90 seconds across 102 iterations.**

Mean Accuracy:          0.68550456 ± 0.02330337

Mean Weighted F1-Score: 0.66898962 ± 0.02293799

Mean Micro F1-Score:    0.68550456 ± 0.02330337

Mean Macro F1-Score:    0.35134204 ± 0.01969395

Each fold had 539 entries for training and 269 for validation.

**TEST SET RESULTS**

In [8]:
from sklearn.metrics import classification_report, confusion_matrix

start_time_2 = datetime.now()

# Train the model
model_stacking.fit(train_val_X, train_val_y)

end_time_2 = datetime.now()
elapsed_time_2 = end_time_2 - start_time_2

print(f"The training of model_stacking with {len(train_val_X)} data points took {elapsed_time_2.total_seconds():.4f} seconds.\n")

# Predict on the test set
test_pred_y = model_stacking.predict(test_X)

# Compute and display metrics
print(f"The model classified correctly {sum(test_y == test_pred_y)} entries from a total of {len(test_X)}.\n")

print(f"Accuracy on test set:          {accuracy_score(test_y, test_pred_y)}")
print(f"Weighted F1-Score on test set: {f1_score(test_y, test_pred_y, average='weighted')}\n")

print("F1-Score per class\n")

# Generate classification report
report = classification_report(test_y, test_pred_y, output_dict=True, zero_division=0)

# Display F1-score per class
for class_label, metrics in report.items():
    if isinstance(metrics, dict) and 'f1-score' in metrics:
        print(f"Class {class_label}: F1-Score = {metrics['f1-score']:.6f}")

print("\nAccuracy per class\n")

# Display F1-score per class
for class_label, metrics in report.items():
    if isinstance(metrics, dict) and 'recall' in metrics:
        print(f"Class {class_label}: Recall = {metrics['recall']:.6f}") # Recall is equivalent to per-class accuracy

/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    5.4s finished
/usr/local/Caskroom/miniconda/base/envs/ulbdm/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to conver

The training of model_stacking with 808 data points took 11.6861 seconds.

The model classified correctly 147 entries from a total of 203.

Accuracy on test set:          0.7241379310344828
Weighted F1-Score on test set: 0.7075128822919287

F1-Score per class

Class 11: F1-Score = 0.333333
Class 13: F1-Score = 0.852941
Class 14: F1-Score = 0.678571
Class 16: F1-Score = 0.000000
Class 17: F1-Score = 0.000000
Class 2: F1-Score = 0.857143
Class 3: F1-Score = 0.000000
Class 4: F1-Score = 0.545455
Class 6: F1-Score = 0.000000
Class 7: F1-Score = 0.000000
Class 9: F1-Score = 0.680851
Class 99: F1-Score = 0.630137
Class macro avg: F1-Score = 0.381536
Class weighted avg: F1-Score = 0.707513

Accuracy per class

Class 11: Recall = 0.200000
Class 13: Recall = 0.906250
Class 14: Recall = 0.633333
Class 16: Recall = 0.000000
Class 17: Recall = 0.000000
Class 2: Recall = 0.875000
Class 3: Recall = 0.000000
Class 4: Recall = 0.562500
Class 6: Recall = 0.000000
Class 7: Recall = 0.000000
Class 9: Rec

**WITH BASE MODELS PARAMS OPTIMIZATION AND CV = SKF(3, SHUFFLE = TRUE)**

The model classified correctly 147 entries from a total of 203.

Accuracy on test set:          0.7241379310344828

Weighted F1-Score on test set: 0.7075128822919287

**F1-Score per class**

Class 11: F1-Score = 0.333333

Class 13: F1-Score = 0.852941

Class 14: F1-Score = 0.678571

Class 16: F1-Score = 0.000000

Class 17: F1-Score = 0.000000

Class 2: F1-Score = 0.857143

Class 3: F1-Score = 0.000000

Class 4: F1-Score = 0.545455

Class 6: F1-Score = 0.000000

Class 7: F1-Score = 0.000000

Class 9: F1-Score = 0.680851

Class 99: F1-Score = 0.630137

Class macro avg: F1-Score = 0.381536

Class weighted avg: F1-Score = 0.707513

**Accuracy per class**

Class 11: Recall = 0.200000

Class 13: Recall = 0.906250

Class 14: Recall = 0.633333

Class 16: Recall = 0.000000

Class 17: Recall = 0.000000

Class 2: Recall = 0.875000

Class 3: Recall = 0.000000

Class 4: Recall = 0.562500

Class 6: Recall = 0.000000

Class 7: Recall = 0.000000

Class 9: Recall = 0.695652

Class 99: Recall = 0.657143

Class macro avg: Recall = 0.377490

Class weighted avg: Recall = 0.724138

In [9]:
print(classification_report(test_y, test_pred_y, zero_division=0))

              precision    recall  f1-score   support

          11       1.00      0.20      0.33         5
          13       0.81      0.91      0.85        64
          14       0.73      0.63      0.68        30
          16       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         2
           2       0.84      0.88      0.86        24
           3       0.00      0.00      0.00         1
           4       0.53      0.56      0.55        16
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           9       0.67      0.70      0.68        23
          99       0.61      0.66      0.63        35

    accuracy                           0.72       203
   macro avg       0.43      0.38      0.38       203
weighted avg       0.71      0.72      0.71       203



In [22]:
# Convert the dictionary to a pandas DataFrame
report_df = pd.DataFrame(report).transpose()

# Display the DataFrame
report_df = report_df.reset_index().rename(columns={'index': 'class'})

# Drop unwanted rows by label
report_df = report_df[~report_df['class'].isin(["accuracy", "macro avg", "weighted avg"])]

print(report_df)

   class  precision    recall  f1-score  support
0     11   1.000000  0.200000  0.333333      5.0
1     13   0.805556  0.906250  0.852941     64.0
2     14   0.730769  0.633333  0.678571     30.0
3     16   0.000000  0.000000  0.000000      1.0
4     17   0.000000  0.000000  0.000000      2.0
5      2   0.840000  0.875000  0.857143     24.0
6      3   0.000000  0.000000  0.000000      1.0
7      4   0.529412  0.562500  0.545455     16.0
8      6   0.000000  0.000000  0.000000      1.0
9      7   0.000000  0.000000  0.000000      1.0
10     9   0.666667  0.695652  0.680851     23.0
11    99   0.605263  0.657143  0.630137     35.0


In [23]:
# Count distinct values
train_counts = train_val_y.value_counts()

# Convert to DataFrame with desired columns
train_counts_df = train_counts.reset_index()
train_counts_df.columns = ['class', 'train_count']

# Merge value_counts_df with report_df on the 'class' column
merged_df = pd.merge(train_counts_df, report_df, on="class", how="left")
merged_df['support'] = merged_df['support'].astype(int)

print(merged_df)

   class  train_count  precision    recall  f1-score  support
0     13          254   0.805556  0.906250  0.852941       64
1     99          140   0.605263  0.657143  0.630137       35
2     14          119   0.730769  0.633333  0.678571       30
3      2           95   0.840000  0.875000  0.857143       24
4      9           94   0.666667  0.695652  0.680851       23
5      4           62   0.529412  0.562500  0.545455       16
6     11           21   1.000000  0.200000  0.333333        5
7     17            8   0.000000  0.000000  0.000000        2
8      6            5   0.000000  0.000000  0.000000        1
9      3            4   0.000000  0.000000  0.000000        1
10     7            3   0.000000  0.000000  0.000000        1
11    16            3   0.000000  0.000000  0.000000        1


In [24]:
# Filter for correct predictions
correct_predictions = test_y[test_y == test_pred_y]

# Count correct predictions for each class
right_pred_counts = correct_predictions.value_counts()

# Convert to DataFrame with desired columns
right_pred_df = right_pred_counts.reset_index()
right_pred_df.columns = ['class', 'right_pred']

# Merge right_pred_df with merged_df on the 'class' column
final_performance_df = pd.merge(merged_df, right_pred_df, on="class", how="left")

# Replace NaN values with 0
final_performance_df = final_performance_df .fillna(0)

# Display the resulting DataFrame
print(final_performance_df)

   class  train_count  precision    recall  f1-score  support  right_pred
0     13          254   0.805556  0.906250  0.852941       64          58
1     99          140   0.605263  0.657143  0.630137       35          23
2     14          119   0.730769  0.633333  0.678571       30          19
3      2           95   0.840000  0.875000  0.857143       24          21
4      9           94   0.666667  0.695652  0.680851       23          16
5      4           62   0.529412  0.562500  0.545455       16           9
6     11           21   1.000000  0.200000  0.333333        5           1
7     17            8   0.000000  0.000000  0.000000        2           0
8      6            5   0.000000  0.000000  0.000000        1           0
9      3            4   0.000000  0.000000  0.000000        1           0
10     7            3   0.000000  0.000000  0.000000        1           0
11    16            3   0.000000  0.000000  0.000000        1           0


**SAVE AND EXPORT RESULTS**

In [27]:
"""
# Create DataFrame for Scores
accuracy_stacking_params_df = pd.DataFrame({'score': accuracy_scores})
weighted_f1_stacking_params_df = pd.DataFrame({'score': weighted_f1_scores})
micro_f1_stacking_params_df = pd.DataFrame({'score': micro_f1_scores})
macro_f1_stacking_params_df = pd.DataFrame({'score': macro_f1_scores})

# Export the DataFrame to a CSV file
accuracy_stacking_params_df .to_csv('accuracy_stacking_params.csv', index=False)
weighted_f1_stacking_params_df.to_csv('weighted_f1_stacking_params.csv', index=False)
micro_f1_stacking_params_df.to_csv('micro_f1_stacking_params.csv', index=False)
macro_f1_stacking_params_df.to_csv('macro_f1_stacking_params.csv', index=False)
"""

"\n# Create DataFrame for Scores\naccuracy_stacking_params_df = pd.DataFrame({'score': accuracy_scores})\nweighted_f1_stacking_params_df = pd.DataFrame({'score': weighted_f1_scores})\nmicro_f1_stacking_params_df = pd.DataFrame({'score': micro_f1_scores})\nmacro_f1_stacking_params_df = pd.DataFrame({'score': macro_f1_scores})\n\n# Export the DataFrame to a CSV file\naccuracy_stacking_params_df .to_csv('accuracy_stacking_params.csv', index=False)\nweighted_f1_stacking_params_df.to_csv('weighted_f1_stacking_params.csv', index=False)\nmicro_f1_stacking_params_df.to_csv('micro_f1_stacking_params.csv', index=False)\nmacro_f1_stacking_params_df.to_csv('macro_f1_stacking_params.csv', index=False)\n"

In [29]:
"""
# Export the DataFrame with the final performance scores
final_performance_df.to_csv('final_performance.csv', index=False)
"""